In [1]:
# Instalación de librerias
!pip install mediapipe opencv-python
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 15.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.8 which is incompatible.
ydf 0.12.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 4.25.8 which is inco

In [2]:
# Importamos paquetes necesarios
import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt
import gradio as gr
from scipy.spatial import distance as dist

In [5]:
# Índices de los puntos clave de los ojos según MediaPipe
OJOS_IZQ = [362, 385, 387, 263, 373, 380]
OJOS_DER = [33, 160, 158, 133, 153, 144]

# Función para calcular EAR
def calcular_ear(ojos):
    A = dist.euclidean(ojos[1], ojos[5])
    B = dist.euclidean(ojos[2], ojos[4])
    C = dist.euclidean(ojos[0], ojos[3])
    return (A + B) / (2.0 * C)

# Función principal para procesar la imagen
def malla_facial_y_ear(imagen):
    # Conversión de imagen a RGB
    imagen_rgb = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB)

    # Inicialización de MediaPipe
    mp_face_mesh = mp.solutions.face_mesh
    face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1)

    # Procesamiento de la imagen
    resultado = face_mesh.process(imagen_rgb)

    # Dibujo de landmarks y cálculo EAR si hay rostro
    if resultado.multi_face_landmarks:
        altura, ancho, _ = imagen.shape

        for face_landmarks in resultado.multi_face_landmarks:
            # Dibujo de malla facial
            mp.solutions.drawing_utils.draw_landmarks(
                image=imagen,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_TESSELATION,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp.solutions.drawing_utils.DrawingSpec(color=(0, 255, 0), thickness=1, circle_radius=1)
            )

            # Extraer puntos de los ojos
            puntos_ojos_izq = []
            puntos_ojos_der = []

            for idx in OJOS_IZQ:
                x = int(face_landmarks.landmark[idx].x * ancho)
                y = int(face_landmarks.landmark[idx].y * altura)
                puntos_ojos_izq.append((x, y))

            for idx in OJOS_DER:
                x = int(face_landmarks.landmark[idx].x * ancho)
                y = int(face_landmarks.landmark[idx].y * altura)
                puntos_ojos_der.append((x, y))

            if len(puntos_ojos_izq) == 6 and len(puntos_ojos_der) == 6:
                ear_izq = calcular_ear(puntos_ojos_izq)
                ear_der = calcular_ear(puntos_ojos_der)
                ear_prom = (ear_izq + ear_der) / 2.0

                # Mostrar valores EAR sobre la imagen
                cv2.putText(imagen, f"EAR Izq: {ear_izq:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                cv2.putText(imagen, f"EAR Der: {ear_der:.2f}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                cv2.putText(imagen, f"EAR Prom: {ear_prom:.2f}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

    return imagen

# Crear la interfaz de Gradio
iface = gr.Interface(
    fn=malla_facial_y_ear,
    inputs=gr.Image(type="numpy"),
    outputs=gr.Image(type="numpy"),
    title="Face Mesh + EAR (para detección de somnolencia)",
    description="Sube una imagen para detectar landmarks y calcular el EAR promedio con MediaPipe."
)

# Lanzamiento
iface.launch(debug=True)


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://90d0609119b1cf4181.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2191, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 1702, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://90d0609119b1cf4181.gradio.live
